# 0. Imports

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import *
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras import activations as Ac
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
#from tensorflow.keras import layers

# 1. Setup Dataframe

In [2]:
image_folder_path = 'data/classFolders'

In [3]:
data = []
for class_name in os.listdir(image_folder_path):
    class_folder = os.path.join(image_folder_path, class_name)
    if os.path.isdir(class_folder):
        for filename in os.listdir(class_folder):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                relative_path = os.path.join(class_name, filename)
                data.append({'filename': relative_path, 'family_name': class_name})

complete_df = pd.DataFrame(data)

In [4]:
complete_df.head()

,filename,family_name
0,Boeing_787/Boeing 787_0178.jpg,Boeing_787
1,Boeing_787/Boeing 787_0144.jpg,Boeing_787
2,Boeing_787/Boeing 787_0150.jpg,Boeing_787
3,Boeing_787/Boeing 787_0187.jpg,Boeing_787
4,Boeing_787/Boeing 787_0193.jpg,Boeing_787


In [5]:
complete_df['family_name'].value_counts()

family_name
Boeing_787        200
A330              200
Boeing_737_MAX    200
A340              200
Boeing_767        200
ATR-72            200
Boeing_757        200
Boeing_747        200
CRJ-700           200
Boeing_777        200
A350              200
A320              200
A380              200
Dash_8            200
Boeing_737        200
Embraer_E-Jet     200
Name: count, dtype: int64

In [6]:
encoder = LabelEncoder()
complete_df['label'] = encoder.fit_transform(complete_df['family_name'])

In [9]:
complete_df.head()

,filename,family_name,label
0,Boeing_787/Boeing 787_0178.jpg,Boeing_787,12
1,Boeing_787/Boeing 787_0144.jpg,Boeing_787,12
2,Boeing_787/Boeing 787_0150.jpg,Boeing_787,12
3,Boeing_787/Boeing 787_0187.jpg,Boeing_787,12
4,Boeing_787/Boeing 787_0193.jpg,Boeing_787,12


# 2. Split Dataframe

In [10]:
train, df_temp = train_test_split(
    complete_df,
    test_size=0.3,
    stratify=complete_df['label'],
    random_state=42
)

val, test = train_test_split(
    df_temp,
    test_size=0.5,
    stratify=df_temp['label'],
    random_state=42
)

print("Train:", train.shape)
print("Val:", val.shape)
print("Test:", test.shape)


Train: (2240, 3)
Val: (480, 3)
Test: (480, 3)


In [11]:
def build_paths(df, base_path):
    return df.filename.apply(lambda x: os.path.join(base_path, x))

In [12]:
train_paths = build_paths(train, image_folder_path)
val_paths = build_paths(val, image_folder_path)
test_paths = build_paths(test, image_folder_path)

In [13]:
train_labels = to_categorical(train.label)
print(train_labels.shape)

val_labels = to_categorical(val.label)
print(val_labels.shape)

test_labels = to_categorical(test.label)
print(test_labels.shape)

(2240, 16)
(480, 16)
(480, 16)


In [14]:
#temporarily split traning set so that we can augment only 70%
train_aug, train_plain = train_test_split(
    train,
    test_size=0.3,
    stratify=train['label'],
    random_state=42
)

In [15]:
train_aug_paths = train_aug.filename.apply(lambda x: os.path.join(image_folder_path, x))
train_plain_paths = train_plain.filename.apply(lambda x: os.path.join(image_folder_path, x))

train_aug_labels = tf.keras.utils.to_categorical(train_aug['label'])
train_plain_labels = tf.keras.utils.to_categorical(train_plain['label'])

# 3. Build Model

## 3.1 Helper functions

In [16]:
def decode_image(filename, label=None, image_size=(224, 224)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)

    if label is None:
        return image
    else:
        return image, label

In [17]:
def downsample(image, min_scale=0.3, max_scale=0.7):
    scale = tf.random.uniform([], min_scale, max_scale)
    h, w = tf.shape(image)[0], tf.shape(image)[1]
    new_h = tf.cast(tf.cast(h, tf.float32) * scale, tf.int32)
    new_w = tf.cast(tf.cast(w, tf.float32) * scale, tf.int32)

    image = tf.image.resize(image, (new_h, new_w))
    image = tf.image.resize(image, (h, w))
    return image

In [18]:
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.1)
    tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    image = tf.image.random_crop(image, size=[180, 180, 3])
    image = tf.image.resize(image, [224, 224])
    image = downsample(image)

    noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.03)
    image = tf.clip_by_value(image + noise, 0.0, 1.0)

    return image, label

## 3.2 Model

In [19]:
batch_size = 16
epoch_count = 50
AUTO = tf.data.AUTOTUNE

def mish(x):
    return x * K.tanh(Ac.softplus(x))

In [20]:
train_aug_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_aug_paths, train_aug_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(augment, num_parallel_calls=AUTO)
)

train_plain_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_plain_paths, train_plain_labels))
    .map(decode_image, num_parallel_calls=AUTO)
)


In [21]:
train_dataset = (
    train_aug_dataset
    .concatenate(train_plain_dataset)
    .shuffle(2048)
    .repeat()
    .batch(batch_size)
    .prefetch(AUTO)
)

val_dataset = (tf.data.Dataset
        .from_tensor_slices((val_paths, val_labels))
        .map(decode_image, num_parallel_calls=AUTO)
        .batch(batch_size)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices((test_paths, test_labels))
    .map(decode_image, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

In [22]:
def Train_model(model, batch_size, EPOCHS):

    n_steps = train_labels.shape[0] // batch_size

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor = 'val_loss',
        factor = 0.5,
        patience = 3,
        verbose = 1,
        min_lr = 0.0001)

    early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
    )

    Model = model
    history = Model.fit(
        train_dataset,
        steps_per_epoch = n_steps,
        epochs = epoch_count,
        validation_data = val_dataset,
        callbacks=[early_stopping],
        verbose = 1)

    return Model

In [23]:
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation=mish),
    BatchNormalization(),
    Dropout(0.5),
    Dense(512, activation=mish),
    BatchNormalization(),
    Dropout(0.5),
    Dense(16, activation='softmax')
])

optimizer = SGD()#learning_rate=0.01, momentum=0.9, nesterov=True

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ densenet201 (Functional)        │ (None, 7, 7, 1920)     │    18,321,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1920)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     1,967,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │         8,208 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,828,240 (79.45 MB)

 Trainable params: 20,596,112 (78.57 MB)

 Non-trainable params: 232,128 (906.75 KB)

# 4. Train and test model

In [24]:
print('Training')
model = Train_model(model, batch_size, epoch_count)

Training
Epoch 1/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 448s 3s/step - accuracy: 0.0930 - loss: 3.9826 - val_accuracy: 0.1562 - val_loss: 3.1954
Epoch 2/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 427s 3s/step - accuracy: 0.2179 - loss: 2.9487 - val_accuracy: 0.2771 - val_loss: 2.2401
Epoch 3/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 432s 3s/step - accuracy: 0.3805 - loss: 2.0875 - val_accuracy: 0.3979 - val_loss: 2.1235
Epoch 4/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 1276s 9s/step - accuracy: 0.5071 - loss: 1.6180 - val_accuracy: 0.5625 - val_loss: 1.4053
Epoch 5/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 380s 3s/step - accuracy: 0.6045 - loss: 1.3347 - val_accuracy: 0.5813 - val_loss: 1.3207
Epoch 6/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 448s 3s/step - accuracy: 0.6565 - loss: 1.0364 - val_accuracy: 0.6812 - val_loss: 1.0144
Epoch 7/50
 28/140 ━━━━━━━━━━━━━━━━━━━━ 5:42 3s/step - accuracy: 0.7149 - loss: 0.7851

KeyboardInterrupt: 

In [ ]:
loss, acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {acc:.4f}")

In [ ]:
model.save('path/modelname.keras')